¿Que hace este script?
* Calcula para cada dispositivo de red: trafico.totalAlarmas, trafico.sumaAlarmas(duracion)  

In [8]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [9]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [10]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

* indice es para hacer la inserción.
* indice_control se usa para controlar el poblado del indice

In [11]:
indice = parametros.mintic_concat_index
indice_control = parametros.mintic_control

### Función para generar JSON compatible con ES

In [12]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO.

In [13]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [14]:
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla['trafico.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['trafico.location']=datos_semilla['trafico.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'trafico.nombreMunicipio'
                                              , 'nombre_departamento' : 'trafico.nombreDepartamento'
                                              , 'nombre_centro_pob': 'trafico.localidad'
                                              , 'nombreSede' : 'trafico.nomCentroDigital'
                                              , 'energiadesc' : 'trafico.sistemaEnergia'
                                              , 'COD_ISO' : 'trafico.codISO'
                                              , 'id_Beneficiario' : 'trafico.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR.

In [15]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac","ip","ap_group","name","status"]  
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_dev = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    exit()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [16]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

In [17]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

Se borran duplicados de datos_dev por mac

In [ ]:
datos_dev = datos_dev.drop_duplicates('mac')

### Cambiando nombre de campos y generando location

In [18]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'trafico.macRed','ap_group': 'trafico.apGroup'
                                        , 'ip': 'trafico.IP'
                                        , 'mac' : 'trafico.macRed'
                                        , 'name' : 'trafico.deviceName'
                                        , 'status' : 'trafico.status.macRed'})

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [20]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["trafico.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_trafico_alarmas"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['trafico.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-05-01 00:00:00


## Se lee la información de cambium alarm history

Se toma el mismo rango de fechas definido al principio de este script. Tra e los campos:
* mac del dispositivo de red
* time_raised es la fecha y hora cuando se levantó la alerta.
* duration (duración de la falla)

### Procesamiento de alarmas

In [200]:
def traeAlarmas(fecha_max,fecha_tope):
    total_docs = 100000
    response = es.search(
        index= parametros.cambium_a_h_index,
        body={
                "_source": ["mac","time_raised","duration","radio.tx_bytes"]
              , "query": {
                  "range": {
                    "time_raised": {
                      "gte": fecha_max,
                      "lt": fecha_tope
                    }
                  }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 

### Lanzando  ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [201]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_alarmas = traeAlarmas(fecha_max_mintic,fecha_tope_mintic)

if datos_alarmas is None or datos_alarmas.empty:
    while (datos_alarmas is None or datos_alarmas.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        datos_alarmas = traeAlarmas(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# 1. Alarmas a indice principal

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se decaran en este, no se insertarán

In [203]:
use_these_keys = ['trafico.nomCentroDigital', 
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.nombreDepartamento', 
                  'trafico.codISO', 
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio', 
                  'trafico.idBeneficiario',
                  'trafico.apGroup', 
                  'trafico.IP', 
                  'trafico.deviceName',
                  'trafico.macRed', 
                  'trafico.location', 
                  'trafico.totalAlarmas',
                  'trafico.sumaAlarmas',
                  'trafico.totales.fechaControl',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.minuto', 
                  'trafico.status.macRed',
                    'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Alarmas-'+document['trafico.siteID'] + '-' + document['trafico.macRed'] + '-' +document['trafico.totales.fechaControl']}",
                "_source": filterKeys(document),
            }

* Se calcula la fecha poniendo a cero los segundos y el segundo dígito de los minutos. Esto para agrupar por bloques de 10 minutos
* Se cuentan las fallas para cada mac y fecha
* Se generan campos nuevos para cada componente de fecha
* Se cruza con datos_dev para saber el AP group
* Se cruza con datos semilla
* Se agrega el instante de tiempo para la inserción en indice
* Se escribe en indice

In [204]:
try:
    datos_alarmas['fecha_control'] = datos_alarmas["time_raised"].str[0:-4] + '0:00'
    datos_alarmas=datos_alarmas[['mac','fecha_control','duration']].groupby(['mac','fecha_control']).agg(['sum','count']).reset_index()
    datos_alarmas.columns = datos_alarmas.columns.droplevel(1)
    datos_alarmas.columns = ['trafico.macRed','fecha_control','trafico.sumaAlarmas','trafico.totalAlarmas']
    datos_alarmas.fillna({'trafico.sumaAlarmas': 0,'trafico.totalAlarmas': 0}, inplace=True)
    
    datos_alarmas["trafico.totales.fecha"] = datos_alarmas["fecha_control"].str.split(" ", n = 1, expand = True)[0]
    datos_alarmas["trafico.totales.hora"] = datos_alarmas["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
    datos_alarmas["trafico.totales.minuto"] = datos_alarmas["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
    datos_alarmas["trafico.totales.anyo"] = datos_alarmas["trafico.totales.fecha"].str[0:4]
    datos_alarmas["trafico.totales.mes"] = datos_alarmas["trafico.totales.fecha"].str[5:7]
    datos_alarmas["trafico.totales.dia"] = datos_alarmas["trafico.totales.fecha"].str[8:10]
    ### Renombrado de campos
    datos_alarmas.rename(columns={'site_id': 'trafico.siteID'
                             ,'fecha_control' : 'trafico.totales.fechaControl'}, inplace=True)
    datos_alarmas = pd.merge(datos_alarmas,datos_dev, on = 'trafico.macRed', how = 'inner')
    datos_alarmas = pd.merge(datos_alarmas,datos_semilla, on = 'site_id', how = 'inner')
    datos_alarmas.rename(columns={'site_id': 'trafico.siteID'}, inplace=True)
    datos_alarmas['nombreDepartamento'] = datos_alarmas['trafico.nombreDepartamento']
    datos_alarmas['nombreMunicipio'] = datos_alarmas['trafico.nombreMunicipio']
    datos_alarmas['idBeneficiario'] = datos_alarmas['trafico.idBeneficiario']
    datos_alarmas['fecha'] = datos_alarmas['trafico.totales.fecha']
    datos_alarmas['anyo'] = datos_alarmas['trafico.totales.anyo']
    datos_alarmas['mes'] = datos_alarmas['trafico.totales.mes']
    datos_alarmas['dia'] = datos_alarmas['trafico.totales.dia']
    datos_alarmas['@timestamp'] = now.isoformat()
    salida = helpers.bulk(es, doc_generator(datos_alarmas))
    print("Fecha: ", now,"- Datos Alarmas en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No hay registrps por insertar")
    

Fecha:  2021-05-23 22:57:21.894822 - Datos Alarmas en indice principal: 2


### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [14]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_trafico_alarmas',
        body = { 'jerarquia_trafico_alarmas': 'trafico_alarmas','trafico.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-05-01 00:00:00
